In [11]:
import sys
from pathlib import Path

# Ubicación del notebook
NOTEBOOK_DIR = Path.cwd()

# Raíz del proyecto = subir un nivel desde Notebooks/
PROJECT_ROOT = NOTEBOOK_DIR.parent

# Añadir raíz del proyecto al sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Proyecto raíz detectado:", PROJECT_ROOT)

Proyecto raíz detectado: c:\Users\cathe\Didier Jesus\EduFinance\EduFinance_Simulator


In [12]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from pathlib import Path
from utils.paths import DATA_DIR, FIG_DIR
from utils.loader import load_csv, save_csv

# Directorio base donde están las series pivotadas
SERIES_DATA_DIR = DATA_DIR / "time_series"

# Directorios de figuras
TESTS_FIG_DIR = FIG_DIR / "tests"
ADF_DIR = TESTS_FIG_DIR / "adf"
ACF_DIR = TESTS_FIG_DIR / "acf"
PACF_DIR = TESTS_FIG_DIR / "pacf"
ROLLMEAN_DIR = TESTS_FIG_DIR / "roll_mean"
ROLLSTD_DIR = TESTS_FIG_DIR / "roll_std"

for folder in [TESTS_FIG_DIR, ADF_DIR, ACF_DIR, PACF_DIR, ROLLMEAN_DIR, ROLLSTD_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

print("📁 SERIES_DATA_DIR:", SERIES_DATA_DIR)
print("📁 TESTS_FIG_DIR:", TESTS_FIG_DIR)


📁 SERIES_DATA_DIR: C:\Users\cathe\Didier Jesus\EduFinance\EduFinance_Simulator\data\time_series
📁 TESTS_FIG_DIR: C:\Users\cathe\Didier Jesus\EduFinance\EduFinance_Simulator\figures\tests


In [13]:
""""Cargar las series de tiempo """

prices = load_csv(SERIES_DATA_DIR / "prices.csv")
log_diff = load_csv(SERIES_DATA_DIR / "log_diff.csv")
returns = load_csv(SERIES_DATA_DIR / "returns.csv")
log_prices = load_csv(SERIES_DATA_DIR / "log_prices.csv")
return_squared = load_csv(SERIES_DATA_DIR / "returns_sq.csv")
roll_mean_21 = load_csv(SERIES_DATA_DIR / "roll_mean_21.csv")
roll_std_21 = load_csv(SERIES_DATA_DIR / "roll_std_21.csv")

tickers = [t for t in prices.columns if t.lower() != "date"]
tickers



['BTC-USD', 'EUNL.DE', 'QQQ', 'TSLA', 'V', 'VOO', 'XAR', 'XRP-USD']

In [14]:
"""Aplicamos la prueba ADF"""

# --- Función auxiliar para obtener p-value de ADF ---
def adf_pvalue(series: pd.Series):
    series = series.dropna()
    return adfuller(series)[1]  # p-value

adf_table = []

for t in tickers:   
    row = {'ticker': t,}

    row["p_prices"] = adf_pvalue(prices[t])
    row["p_log_prices"] = adf_pvalue(log_prices[t])
    row["p_log_diff"] = adf_pvalue(log_diff[t])
    row["p_returns"] = adf_pvalue(returns[t])
    row["p_return_squared"] = adf_pvalue(return_squared[t])

    adf_table.append(row)
adf_df = pd.DataFrame(adf_table).set_index('ticker')
save_csv(adf_df, ADF_DIR / "adf_results.csv")
adf_df

Archivo guardado en: C:\Users\cathe\Didier Jesus\EduFinance\EduFinance_Simulator\figures\tests\adf\adf_results.csv


,p_prices,p_log_prices,p_log_diff,p_returns,p_return_squared
ticker,,,,,
BTC-USD,0.926876,0.819959,0.000000e+00,0.000000e+00,1.865260e-28
EUNL.DE,0.945079,0.823829,1.752700e-23,1.178063e-23,5.029919e-07
QQQ,0.976539,0.855836,1.587510e-26,1.071910e-26,4.229583e-15
TSLA,0.483941,0.700882,0.000000e+00,0.000000e+00,1.667427e-09
V,0.923996,0.620011,1.348183e-25,6.581433e-26,4.796102e-08
VOO,0.985270,0.889684,1.099550e-25,6.275930e-26,4.880530e-11
XAR,0.989074,0.868061,2.301547e-22,1.076851e-22,2.025670e-08
XRP-USD,0.730829,0.251687,2.583331e-18,8.322642e-18,6.430455e-11


In [16]:
"""
Prueba ACF

¿Qué significa la ACF?

Si las autocorrelaciones decaen lentamente → la serie no es estacionaria
Si se cortan abruptamente → modelo MA
Si el decaimiento es suave y estable → modelo AR
Si hay patrones → volatilidad → posible GARCH

Ejemplos:
Retornos financieros → ACF casi en cero después del lag 1
Precios crudos → autocorrelación fuerte (tendencia) → no estacionarios
Volatilidad (returns^2) → autocorrelación prolongada → clustering de volatilidad

La prueba sera realizada sobre la serie de log-diff
"""

acf_series = log_diff.copy()

for t in tickers:
    series = acf_series[t]

    fig = plt.figure(figsize=(10, 6))
    plot_acf(series, lags=40, ax=plt.gca())
    plt.title(f"ACF de log-diff: {t}")
    plt.tight_layout()

    fig.savefig(ACF_DIR / f"{t}_acf.png", dpi=300, bbox_inches='tight')
    plt.close(fig)  # Cerrar la figura para liberar memoria

ACF_DIR

WindowsPath('C:/Users/cathe/Didier Jesus/EduFinance/EduFinance_Simulator/figures/tests/acf')

In [17]:
"""
Prueba PACF

¿Qué dice la PACF?

Si los primeros rezagos (lags) son significativos (superan las bandas azules), indica componente AR.
Si solo el lag 1 es significativo → AR(1)
Si lags 1 y 2 son significativos → AR(2)
Si muchos lags son significativos → proceso AR más complejo.
"""

pacf_series = log_diff.copy()


for t in tickers:
    series = pacf_series[t]

    fig = plt.figure(figsize=(10, 6))
    plot_pacf(series, lags=40, ax=plt.gca(), method='ywm')
    plt.title(f"PACF de log-diff: {t}")
    plt.tight_layout()

    fig.savefig(PACF_DIR / f"{t}_pacf.png", dpi=300, bbox_inches='tight')
    plt.close(fig)  # Cerrar la figura para liberar memoria

In [22]:
"""
Rolling mean para cada ticker
"""

for t in tickers:
    fig, ax = plt.subplots(figsize=(12,4))

    ax.plot(prices.index, prices[t], label="Precio", alpha=0.7)
    ax.plot(roll_mean_21.index, roll_mean_21[t], label="Media móvil (21)", alpha=0.9)

    ax.set_title(f"{t} — Precio y Media Móvil (21 días)")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Precio")
    ax.legend()
    ax.grid(True)

    #guardar figura
    fig.savefig(ROLLMEAN_DIR / f"{t}_roll_mean_21.png", dpi=300, bbox_inches='tight')
    plt.close(fig)  # Cerrar la figura para liberar memoria


In [19]:
"""Rolling std"""

for t in tickers:
    fig, ax = plt.subplots(figsize=(12, 6))

    ax.plot(roll_std_21.index, roll_std_21[t], lw = 1.2)
    ax.set_title(f"Rolling Std (21 días) de {t}")
    ax.set_xlabel("Fecha")
    ax.set_ylabel("Rolling Std")
    ax.grid(True, alpha = 0.3)

    #guardar figura
    fig.savefig(ROLLSTD_DIR / f"{t}_roll_std_21.png", dpi=300, bbox_inches='tight')
    plt.close(fig)  # Cerrar la figura para liberar memoria